![Skytruth Logo](skytruth_logo.png)
# Cerulean - Global Monitoring of Oil Pollution with Sentinel-1
## Slick to AIS Association

#### This notebook serves as a step-by-step guide for running the slick/AIS association pipeline for a single input sample.

In [ ]:
from datetime import datetime, timedelta
import os

import geopandas as gpd
import matplotlib.pyplot as plt
import movingpandas as mpd
import numpy as np
import pandas as pd

from utils.associate import associate_ais_to_slicks, slicks_to_curves
from utils.constants import (DATA_DIR, AIS_DIR, SLICK_DIR, TRUTH_FILE,
                             HOURS_BEFORE, NUM_TIMESTEPS,
                             BUF_START, BUF_END, BUF_VEC,
                             WEIGHT_START, WEIGHT_END, WEIGHT_VEC)
from utils.gee import get_s1_tile_layer
from utils.misc import build_time_vec, get_utm_zone
from utils.trajectory import ais_points_to_trajectories, buffer_trajectories

#### Read in sample data - a single S1 scene

In [ ]:
PID = 'S1B_IW_GRDH_1SDV_20210626T051247_20210626T051312_027523_034911_53D5'
ais_file = f'./sample_data/{PID}_ais.geojson'
slicks_file = f'./sample_data/{PID}_slicks.geojson'
oldmodel_file = f'./sample_data/{PID}_oldmodel.csv'

In [ ]:
ais = gpd.read_file(ais_file)
slicks = gpd.read_file(slicks_file)
oldmodel = pd.read_csv(oldmodel_file)

#### Build time vector, reproject to UTM

In [ ]:
collect_time = datetime.strptime(PID.split('_')[4], '%Y%m%dT%H%M%S')
time_vec = build_time_vec(collect_time, HOURS_BEFORE, NUM_TIMESTEPS)

In [ ]:
utm_zone = get_utm_zone(ais)
ais = ais.to_crs(utm_zone)
slicks = slicks.to_crs(utm_zone)

#### Build AIS trajectories along the time vector

In [ ]:
ais_traj = ais_points_to_trajectories(ais, time_vec)

#### Create conic buffers with corresponding weights around AIS trajectories

In [ ]:
ais_buffered, ais_weighted = buffer_trajectories(ais_traj, BUF_VEC, WEIGHT_VEC)

#### Fit a curve for every oil slick

In [ ]:
slicks_clean, slicks_curves = slicks_to_curves(slicks)

#### Associate oil slicks to AIS trajectories

In [ ]:
slick_ais = associate_ais_to_slicks(ais_traj, ais_buffered, ais_weighted, slicks_clean, slicks_curves)

#### Display highest score per slick in the scene

In [ ]:
results = slick_ais.sort_values(
    ['slick_index', 'slick_size', 'total_score'],
    ascending=[True, False, False]
).groupby('slick_index')

In [ ]:
for name, group in results:
    disp = group[['slick_index', 'total_score', 'temporal_score', 'overlap_score', 'frechet_dist', 'traj_id']].head(1)
    display(disp)